In [1]:
import geopandas as gp
from multiprocess import Pool
SF57 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
SF57R = {value : key for (key, value) in SF57.items()}
CRS_REGRID_PARCELS = "EPSG:4326"


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Read TIGER2019 censusblock, and save GEOID + wkt columns to ftr

In [13]:
import requests
def saveCB(sf):    
    state_shp_url = f"https://www2.census.gov/geo/tiger/TIGER2019/TABBLOCK/tl_2019_{sf}_tabblock10.zip"
    if requests.get(state_shp_url).ok: # PING FIRST, check if the file is available
        state_census_tiles = gp.read_file(state_shp_url).to_crs(CRS_REGRID_PARCELS)
        # print("READ TIGER INPUT = ", state_census_tiles.shape)
        state_census_tiles['geom'] = state_census_tiles.geometry.to_wkt()
        # state_census_tiles[['GEOID10', 'geom']] is a pandas DataFrame
        sf_code = SF57[sf]
        state_census_tiles[['GEOID10', 'geom']].to_feather(f'tigerCB2019/{sf_code}.ftr')
        print(f"Completed saving {state_census_tiles.shape} in {sf_code=} to ftr")
        del state_census_tiles
    return
# NOTE: TIGER2019/CENSUSBLOCK has no coverage for UM statefips = 74, neither does Elasticsearch bossdata (no bossdata74 index)    
with Pool(16) as pool:
    pool.map(saveCB, SF57)
    

Completed saving (6507, 17) in sf_code='DC' to ftr
Completed saving (24115, 17) in sf_code='DE' to ftr
Completed saving (25016, 17) in sf_code='HI' to ftr
Completed saving (67578, 17) in sf_code='CT' to ftr
Completed saving (45292, 17) in sf_code='AK' to ftr
Completed saving (69518, 17) in sf_code='ME' to ftr
Completed saving (216007, 17) in sf_code='IA' to ftr
Completed saving (149842, 17) in sf_code='ID' to ftr
Completed saving (238600, 17) in sf_code='KS' to ftr
Completed saving (186211, 17) in sf_code='AR' to ftr
Completed saving (267071, 17) in sf_code='IN' to ftr
Completed saving (201062, 17) in sf_code='CO' to ftr
Completed saving (145247, 17) in sf_code='MD' to ftr
Completed saving (241666, 17) in sf_code='AZ' to ftr
Completed saving (161672, 17) in sf_code='KY' to ftr
Completed saving (204447, 17) in sf_code='LA' to ftr
Completed saving (252266, 17) in sf_code='AL' to ftr
Completed saving (48837, 17) in sf_code='NH' to ftr
Completed saving (291086, 17) in sf_code='GA' to ftr
C

In [14]:
ls tigerCB2019 | wc -l

56


In [16]:
ls -lhS tigerCB2019 | head -5

total 6.9G
-rw-r--r-- 1 nhat nhat 455M Jun 14 13:32 TX.ftr
-rw-r--r-- 1 nhat nhat 417M Jun 14 13:31 CA.ftr
-rw-r--r-- 1 nhat nhat 266M Jun 14 13:31 PA.ftr
-rw-r--r-- 1 nhat nhat 232M Jun 14 13:31 VA.ftr
